LOADING THE ELECTRICITY DEMAND DATA

In [ ]:
import pandas as pd
import glob
from datetime import timedelta

#path to the drive folder
path = "Demand_data"
#instruction for all files in the folder that end with csv to be appended onto a list using the glob module
all_csv_files = glob.glob(path + "/*.csv")
#creating empty demand list onto which we will add the read csv files as individual lists of files, imagine like in a drawer
demand_list = []

#loop to read through all the filenames that we have created using glob module - those that end with csv
for filename in all_csv_files:
    #reading individual csv file in each cycle
    df = pd.read_csv(filename, index_col=None, header = 0)
    #extract that specific's file first settlement date as the start date - using iloc
    date = pd.to_datetime(df['SETTLEMENT_DATE'].iloc[0])
    #create an empty list onto which we will be appending all the generated timestamps
    timestamps = []
    #for loop to generate the timestamps of corresponding length to the dataframe and in timesteps of 30 minutes then append that data to the timestamps list
    for i in range(0, len(df)*30, 30):
      timestamps.append(date + timedelta(minutes=i))
    #adding the new timestamps lists as an extra column for time identification in the new dataframe
    df['date'] = timestamps
    #appending the new dataframe (with each cycle) into the list of the dataframes
    demand_list.append(df)

#Now concatenate all the lists into one dataframe
df_demand = pd.concat(demand_list, axis = 0, ignore_index=True)

#Filter for only pre-covid demand
df_demand = df_demand[df_demand['date'] < '2020-01-01 00:00:00']
df_demand = df_demand[df_demand['date'] > '2014-12-31 23:30:00']

#Reset the index to identify the split points better later
df_demand = df_demand.reset_index(drop = True)

#Terminal prints to confirm filtered dataframe start and end dates
print(f'First day:', df_demand['date'].iloc[0])
print(f'Last day:', df_demand['date'].iloc[-1])
print(f'dataframe length:', len(df_demand))


EXTRATING AND ADJUSTING THE DEMAND DATA FOR MODEL PREPARATION

In [ ]:
import numpy as np
from numpy import array

#extract demand data from dataframe and convert it into a Gigawatt list (157776 elements)
demand = list(map(float, (df_demand['ND']/1000)))

#Terminal print to confirm list same length as dataframe and data in GW instead of MW
print(f'demand data length:', len(demand))
print(f'first ten demand elements in GW before normalisation:', demand[:10])
print(f'last ten demand elements in GW before normalisation:', demand[-10:])

Subsection: Identify the split point

In [ ]:
#First three years for training ('15, '16, 17)
split_point_one = df_demand[df_demand['date'] == '2016-01-01 00:00:00'].index[0]

#'18 set aside as the valudation set and 2019 for the test set
split_point_two = df_demand[df_demand['date'] == '2017-01-01 00:00:00'].index[0]

#'18 set aside as the valudation set and 2019 for the test set. Just for quick testing
split_point_three = df_demand[df_demand['date'] == '2018-01-01 00:00:00'].index[0]

#Terminal print to confirm split points
print(f'The split point is at index', split_point_one, 'while the second one is:', split_point_two)
print(f'the third split point is', split_point_three)

Subsection: perform the split

In [ ]:
#apply that split point on the list of demand data careful to include the slight overlap in test set to ensure no skip in predictions
#set the training window - 2 weeks (2 weeks *7 days *48 settlement period) input to map out to one day output (48 settlement periods)
n_steps = 2*7*48
train, val, test = demand[0:split_point_one], demand[(split_point_one - n_steps):(split_point_two)], demand[(split_point_two - n_steps):split_point_three]
print(f'Training set length is', len(train), 'validation set is:', len(val),'while test set length is', len(test))

Subsection: normalising the data using the minmax scaler algorithm. Note that I tried running without normalisation and the model training quickly runs into a nan loss within the first few training batches. Also, the other program file explores the use of the standard scaler as an alternative. Normalisation temporarily excluded

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# define min max scaler object
scaler = MinMaxScaler()

# fit data. This identifies the max/ min demand and computes the mean and standard deviation to be used by all the other transformations
print(scaler.fit(np.array(train).reshape(-1, 1)))

#Terminal print to identify what the maximum demand in the training set was
print(scaler.data_max_)

#Now that we have identified, we perform the tranform but the data needs to be converted from list to array then reshaped
train = scaler.transform(np.array(train).reshape(-1, 1))
print(train.shape)

#Since we reshaped the data to a single stretching column, we now reconvert that to a horizontal array
train = train.reshape(-1)
print(f'First 5 elements of transformed training data:', train[0:5])

#Transforming the validation data
val = scaler.transform(np.array(val).reshape(-1,1))
print(val.shape)

#Reconverting validation data to a horizontal array again
val = val.reshape(-1)
print(f'First 5 elements of transformed validation data:', val[0:5])

Now creating the function to reshape the data before feeding it into the model

In [6]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
 #loop through the list and update the days so that instead of sliding the window across one point, its across 48 points (one day)
	for i in range(0, len(sequence), 48):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-48:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix: end_ix+48]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

Subsection: reshape training data for the model

In [ ]:
X_train, y_train = split_sequence(train, n_steps)

#Useful to understanding the new shaping
print('X_train data')
print('------------')
print(X_train.shape)
print(len(X_train[0]))
print(X_train[0][0:6])

print('y_train data')
print('------------')
print(y_train.shape)
print(len(y_train[0]))
print(y_train[0][0:6])

Subsection: reshape the validation data

In [ ]:
X_val, y_val = split_sequence(val, n_steps)

#Useful to understanding the new shaping
print('X_val data')
print('------------')
print(X_val.shape)
print(len(X_val[0]))
print(X_val[0][0:6])

print('y_val data')
print('------------')
print(y_val.shape)
print(len(y_val[0]))
print(y_val[0][0:6])

Subsection: reshaping the training set

In [9]:
#univariate hence n_features = 1
n_features = 1
#This reshapes the data such that if you imagine you are staring at a screen:
#n_features is the number of vertical lines you can draw on that screen
#X_train.shape[1], in this case, 672,is the number of lines you could draw from left to right on the same screen
#X_train.shape[0], in this case 1082 (the number of batched samples) is the number of lines you could draw into or out of the screen
#If you imagine the screen that you are facing being pushed to your right, almost like into a slot, then what we have done is reshape the data
#such that the past 672 demand points are written vertically and being pushed as one long vertical line into the slot and will be mapped to the 48 we have on the other end
#This mapping will happen 139537 times in cycle (epoch) during which the model will try to identify any patterns.
#The learning is then repeated across several epochs
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
print(f'the new x_train shape for the model is', X_train.shape)

the new x_train shape for the model is (351, 672, 1)


Subsection: building the autoregressive model. Note that I had to lower the learning rate by 10 factor 3 (default is 0.0001) to avoid the training running into infinite then nan loss values (which before, was happening by the time we got to the 30th of 4361 batches). I also added the clipvalue to try prevent the same but might be worthwhile to see how the model would perform if we were to remove it.

Updated to include the live plotting

In [ ]:
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from IPython.display import clear_output
from bayes_opt import BayesianOptimization

#CREATING DATAFRAME TO SAVE RELEVANT RESULTS
results_df = pd.DataFrame(columns=["Iteration", "Start Time", "End Time", "unit1", "unit2", "drop1", "drop2", "learn_rate", "step1", "step2", "step3", "rate1", "rate2", "rate3", "Train Loss", "Val Loss", "Train MSE", "Val MSE"])

#SET UP TRAINING PARAMETERS

# Function to calculate root mean squared error (RMSE)
def root_mean_squared_error(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

#set batch_size
batch_size = int(X_train.shape[0]/8)
print(f'Batch size:', batch_size)

def optimize_lstm(unit1, drop1,unit2,drop2,learn_rate,step1,rate1,step2,rate2, step3, rate3):

    #These are some of the hyperparameters that we will be adjusting using bayesian optimisation, though some of them are, actually, parameters in that they stay constant throughout the exploration process. Example, the intial learning rate
    unit1 = int(unit1)
    unit2 = int(unit2)
    step1 = int(step1)
    step2 = int(step2)
    step3 = int(step3)
    initial_learning_rate = learn_rate
    lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay([step1, step2, step3], [learn_rate, rate1,rate2,rate3])
    #What this is saying is this, take, for example, the first iteration being step1   (0,500), learn_rate1(0.0005) - which is now the initial learning rate and rate1(0.0001, 0.001). This is saying that for the first 0th iteration, train with initial learning rate - 0.0005, for the next 500 iterations, train with first element of rate1 - 0.0001 and for the remainder, train with the new rate - 0.001. Include link (ChatGPT Optimising hyperparameter note folder)

    #Here we define the start time
    start_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())


    #This is where we build the actual model. Note unit1 is first_layer_neuron_size, drop_1 is dropout_rate for first dropout layer, unit2 is the number of neurons in the second bidirectional lstm layer then drop2 is the dropout rate for the second dropout layer. So we are adjusting layer1, rate1, layer2,rate2
    model = Sequential([
        tf.keras.layers.Bidirectional(LSTM(unit1, return_sequences = True, input_shape = (n_steps, n_features))),
        tf.keras.layers.Dropout(drop1),
        tf.keras.layers.Bidirectional(LSTM(unit2, return_sequences = False)),
        tf.keras.layers.Dropout(drop2),
        tf.keras.layers.Dense(48)
    ])

    #Now these are some of the training parameters to reduce on the training time. We are telling the model that for each training period, stop once you get to a point where even after 100 consecutive trainings, there is no change in the validation loss
    early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 100)

    #This is now where we bring everything together into the model. First, we built the model architecture, then we defined a few other things to optimise the training such as early stopping then now we bring everything together with compile saying:
    #when training the model, use Adam optimiser to find the best weights, use the following learning rate schedule
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule), loss = root_mean_squared_error, metrics = ['mse'])
    model.build(input_shape = (None, n_steps, n_features))

    history = model.fit(x = X_train,
                        y = y_train,
                        verbose = 2, #just want to see the epoch and the training/ validation losses. Don't care about time logs yet
                        batch_size = batch_size,
                        epochs = 5000,
                        validation_data = [X_val, y_val],
                        callbacks = [early_stop]
                        )

    #Obtaining all the results and saving them to a dictionary
    iteration_results = {
        "Iteration": len(results_df) + 1,
        "Start Time": start_time,
        "End Time": time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()),
        "unit1": unit1,
        "unit2": unit2,
        "drop1": drop1,
        "drop2": drop2,
        "learn_rate": learn_rate,
        "step1": step1,
        "step2": step2,
        "step3": step3,
        "rate1": rate1,
        "rate2": rate2,
        "rate3": rate3,
        "Train Loss": history.history["loss"][-1],
        "Val Loss": history.history["val_loss"][-1],
        "Train MSE": history.history["mse"][-1],
        "Val MSE": history.history["val_mse"][-1]
    }

    # Append iteration results to results_df
    results_df.loc[len(results_df)] = iteration_results

    # Save results to CSV after each iteration
    results_df.to_csv('Bayesian_BiLSTM_analysis.csv', index=False)

    best_val_loss = min(history.history['val_loss'])

    return -best_val_loss


#Dictionary to store the boundaries probably. Note that the way the Bayesian optimiser works is not discrete like with the talos gridsearch option. This is almost continuous

###THIS IS THE SECTION THAT YOU NEED TO ADJUST DEPENDING ON YOUR PARAMETERS
p_bounds = {
    'unit1':(20,128), #first layer neurons - from 1 to 100
    'unit2': (20,256),#second layer neurons - from 1 to 100
    'drop1': (0.05,0.5), #first dropout rate - from 0.05 to 0.5
    'drop2': (0.05,0.5), #second dropout rate
    'learn_rate': (0.0005, 0.005), #there are two initial learning rates. First, start with 0.0005 and follow the schedule. Then restart with 0.005 and follow the schedule
    'step1': (10, 500),
    'step2': (1000, 2000),
    'step3': (2000, 5000),
    'rate1': (0.0001, 0.001),
    'rate2': (0.00005, 0.0005),
    'rate3': (0.000005, 0.00005)
}

#create an object for the optimiser
optimizer = BayesianOptimization(f = optimize_lstm, pbounds = p_bounds, random_state = None, verbose = 2)

#Then apply the .maximize method
#This will generate 15 different sets of hyperparameters in the beginning (init_points)
#Later, it will select a new set of hyperparameters (16th) using the surrogate model and keep doing this another 14 more times (n_iter = 15). In total, this model will run 30 iterations and the results will be saved into the dataframe that I created up there.
optimizer.maximize(init_points = 15, n_iter = 15)

THE END